# Deep learning and AI methods
## Session 3: Feedforward Neural Network and cloth images
* Instructor: [Krzysztof Podgorski](https://krys.neocities.org),  [Statistics, Lund University, LUSEM](https://www.stat.lu.se/)
* For more information visit the [CANVAS class website](https://canvas.education.lu.se/courses/1712).

In this session we learn some basic about *TensorFlow* and *Keras* by solving an example of immage recognition problem. Note that the code is not necessarily complete so it may not be enough just to run the code cells and somme your own programing may be needed on some occasions. 

## Project 1 Building neural networks for classification

This project trains a neural network model to classify images of clothing, like sneakers and shirts. It's okay if you don't understand all the details; this is a fast-paced overview of a complete TensorFlow program with the details explained as you go.

This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Importing tensorflow and keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.15.0


In [18]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images_raw, train_labels), (test_images_raw, test_labels) = fashion_mnist.load_data()

The next part is a repetition from the previous lab to obtain the split of the data for cross-validation and pre-process the data.

In [19]:
# Scaling the data
train_images = train_images_raw / 255.0
test_images = test_images_raw / 255.0

# Shuffle and rearrange data
perm = np.random.permutation(60000)
rearr = perm.reshape(10, 6000)

# Create cross validation set
CV_images = train_images[rearr]
CV_labels = train_labels[rearr]
print(len(CV_images[0]))

# Scale again (why?)
# CV_images = CV_images / 255.0

6000


In [20]:
%%time

# Prepare cross validation set 1
CVin = [list(rearr[0]), list(rearr[1])]
for k in range(8):
    CVin[1] = CVin[1] + list(rearr[k+2]%10)
CVin = [CVin]

# Same for the rest of the sets
for i in range(9):
    C = [list(rearr[i+1]), list(rearr[(i+2)%10])]
    for k in range(8):
        C[1] = C[1] + list(rearr[(i+3+k)%10])
    CVin = CVin + [C]

CPU times: user 85.5 ms, sys: 40.4 ms, total: 126 ms
Wall time: 130 ms


### Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

#### Set up the layers

The basic building block of a neural network is the *layer*. Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as `tf.keras.layers.Dense`, have parameters that are learned during training.

In [21]:
# Set up the model using two hidden layers
model = keras.Sequential([
    keras.layers.Flatten(input_shape = (28, 28)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a two-dimensional array (of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely connected, or fully connected, neural layers. The first `Dense` layer has 128 nodes (or neurons). The second (and last) layer is a 10-node *softmax* layer that returns an array of 10 probability scores that sum to 1. Each node contains a score that indicates the probability that the current image belongs to one of the 10 classes.

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's *compile* step:

* *Loss function* —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
* *Optimizer* —This is how the model is updated based on the data it sees and its loss function.
* *Metrics* —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

In [22]:
# Set up training 'instructions' for the model
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

### Task 1: Train the model

Check the basic info about the model using `model.summary()`. How many parameters is there in the model? How could you count without using this function? Training the neural network model requires the following steps:

1. Feed the training data to the model. In this example, the training data is in the `train_images` and `train_labels` arrays.
2. The model learns to associate images and labels.
3. You ask the model to make predictions about a test set—in this example, the `test_images` array. Verify that the predictions match the labels from the `test_labels` array.

To start training,  call the `model.fit` method — it is so called because it "fits" the model to the training data.

Please, perform fitting the model and analyze its performance. Compare the reported accuracy on the training data to the one on the testing data set. For the latter, use `model.evaluate(test_images,  test_labels, verbose=2)`. Note that the algorithms also give the loss function values. 

Compare the results with the ones obtained by the MGaussianLE method from the previous lab. 

In [24]:
# Check model setup
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [25]:
%%time

# Fit the model
model.fit(train_images, train_labels, epochs = 10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 12s 204us/sample - loss: 0.5006 - acc: 0.8240
Epoch 2/10
60000/60000 [==============================] - 10s 173us/sample - loss: 0.3772 - acc: 0.8627
Epoch 3/10
60000/60000 [==============================] - 12s 192us/sample - loss: 0.3362 - acc: 0.8759
Epoch 4/10
60000/60000 [==============================] - 18s 299us/sample - loss: 0.3127 - acc: 0.8855
Epoch 5/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.2953 - acc: 0.8914
Epoch 6/10
60000/60000 [==============================] - 10s 173us/sample - loss: 0.2811 - acc: 0.8957
Epoch 7/10
60000/60000 [==============================] - 10s 164us/sample - loss: 0.2673 - acc: 0.9005
Epoch 8/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.2573 - acc: 0.9039
Epoch 9/10
60000/60000 [==============================] - 12s 199us/sample - loss: 0.2486 - acc: 0.9062
Epoch 10/10
60000/60000 [================

#### Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [28]:
# Evaluate test data
test_loss_, test_acc = model.evaluate(test_images, test_labels, verbose = 2)
print('\nTest accuracy:', test_acc)
print('\nError rate:', 1 - test_acc)

10000/10000 - 1s - loss: 0.3380 - acc: 0.8834

Test accuracy: 0.8834

Error rate: 0.11659997701644897


*Comment*: To start with, we can note that it's already performing quite a bit better then the ML classifier we built in lab 2 (which had an error rate around 30 %).

### Task 2:

Consider five other designs of neural networks that you would test on the above data set. In your design change the number of layers and the number of neuron per layer but preserve the total number of neurons.


#### Solution
I'll now build five more models, in which I'll slightly change some of the model settings. We can call this changing the model architecture, if we want to use big words. Summary of how I'll change the models:

* Model1: Add one extra 'relu' activaion hidden layer
* Model2: Add four extra 'relu' activation hidden layer
* Model3: Change activation in hidden later in original model to sigmoid
* Model4: Change loss function to ordinary (non-sparse) categorical crossentropy
* Model5: Increase the learning rate of the adam optimizer

Not that I'm not actually fitting them as that seems to be the next task.

In [33]:
# Model 1

# Architecture
model1 = keras.Sequential([
    keras.layers.Flatten(input_shape = (28, 28)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

# Training settings
model1.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [34]:
# Model 2

# Architechture
model2 = keras.Sequential([
    keras.layers.Flatten(input_shape = (28, 28)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

# Training settings
model2.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [35]:
# Model 3

# Architechture
model3 = keras.Sequential([
    keras.layers.Flatten(input_shape = (28, 28)),
    keras.layers.Dense(128, activation = 'sigmoid'),
    keras.layers.Dense(10, activation = 'softmax')
])

# Training settings
model3.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [37]:
# Model 4

# Architechture
model4 = keras.Sequential([
    keras.layers.Flatten(input_shape = (28, 28)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

# Training settings
model4.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [38]:
# Model 5

# Architechture
model5 = keras.Sequential([
    keras.layers.Flatten(input_shape = (28, 28)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(10, activation = 'softmax')
])

# Training settings
opt = keras.optimizers.Adam(learning_rate = 0.01) # Higher learning rate than default
model4.compile(optimizer = opt,
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [40]:
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_19 (Dense)             (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [41]:
model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_21 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_22 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_23 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_24 (Dense)             (None, 10)                1290      
Total params: 151,306
Trainable params: 151,306
Non-trainable params: 0
________________________________________________

In [42]:
model3.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_26 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [43]:
model4.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_29 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [44]:
model5.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_31 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


### Task 3:

Use the 10 fold cross validation method to investigate the performance of the networks designed by you as well as the the one that was originally trained above (for the total of six networks). Select the one that based on the performed cross validation worked best. It can be computationally (and thus timewise) costly operation. Test the approach with smaller sizes to assess the time needed. You may reduce the size of the data if you see that your computers will not handle the task within reasonable time. 

In [45]:
help(model.fit)

Help on method fit in module tensorflow.python.keras.engine.training:

fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False, **kwargs) method of tensorflow.python.keras.engine.sequential.Sequential instance
    Trains the model for a fixed number of epochs (iterations on a dataset).
    
    Arguments:
        x: Input data. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A dict mapping input names to the corresponding array/tensors,
            if the model has named inputs.
          - A `tf.data` dataset. Should return a tuple
         

In [48]:
a = 0
test_loss_list = []
test_acc_list = []

for k in range(10):
    
    # Pick our training and test set needed for the CV procedure
    testCVim = train_images[CVin[k][0]]
    testCVlb = train_labels[CVin[k][0]]
    trainCVim = train_images[CVin[k][1]]
    trainCVlb = train_labels[CVin[k][1]]
    
    # Delete the old model
    del model 
    
    # Redefine model
    model = keras.Sequential([
        keras.layers.Flatten(input_shape = (28, 28)),
        keras.layers.Dense(128, activation = 'relu'),
        keras.layers.Dense(10, activation = 'softmax')
    ])
    
    # Recompile model
    model.compile(optimizer = 'adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy'])
    
    # Fit model
    model.fit(trainCVim, trainCVlb, epochs = 10)
    test_loss, test_acc = model.evaluate(testCVim, testCVlb, verbose = 0)
    
    # Record the results in our lists
    test_loss_list[k] = test_loss
    test_acc_list[k] = test_acc

Train on 54000 samples
Epoch 1/10
54000/54000 [==============================] - 11s 206us/sample - loss: 0.0996 - acc: 0.9688
Epoch 2/10
54000/54000 [==============================] - 10s 191us/sample - loss: 0.0596 - acc: 0.9790
Epoch 3/10
54000/54000 [==============================] - 10s 182us/sample - loss: 0.0521 - acc: 0.9810
Epoch 4/10
54000/54000 [==============================] - 10s 181us/sample - loss: 0.0467 - acc: 0.9827
Epoch 5/10
54000/54000 [==============================] - 11s 197us/sample - loss: 0.0420 - acc: 0.9844
Epoch 6/10
54000/54000 [==============================] - 13s 232us/sample - loss: 0.0393 - acc: 0.9853
Epoch 7/10
54000/54000 [==============================] - 13s 245us/sample - loss: 0.0370 - acc: 0.9863
Epoch 8/10
54000/54000 [==============================] - 11s 196us/sample - loss: 0.0334 - acc: 0.9872
Epoch 9/10
54000/54000 [==============================] - 10s 189us/sample - loss: 0.0317 - acc: 0.9877
Epoch 10/10
54000/54000 [================

IndexError: list assignment index out of range

In [49]:
print('Accuracy:', np.mean(test_loss_list))

Accuracy: nan


/Users/Sara/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Sara/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Task 4:

Train the selected networks on the entire training data. Then test its performance on the testing data set and summariaze the performance. 

### Task 5: 
With the models trained, you can use them to make predictions about some images. Below you see a discussion of predictions based on the original model. Please, complement it with an analogous discussion for the best model out of five you have trained in the cross-validation. 

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

A prediction is an array of 10 numbers. They represent the model's "confidence" that the image corresponds to each of the 10 different articles of clothing. You can see which label has the highest confidence value:

Graph this to inspect the full set of 10 class predictions.

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.


In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.


### Task 6:
Take the reduced data set (16 x 16) from the previous lab and choose the best neural network based on the performance on the full data. Assess the performance of this network on this data set and compare with the MGaussianLikelihood method assessed in the previous lab.  

## Grader box: 

In what follows the grader will put the values according the following check list:

* 1 Have all commands included in a raw notebook been evaluated? (0 or 0.5pt)
* 2 Have all commands been experimented with? (0 or 0.5pt)
* 3 Have all experiments been briefly commented? (0 or 0.5pt)
* 4 Have all tasks been attempted? (0, 0.5, or 1pt)
* 5 How many of the tasks have been completed? (0, 0.5, or 1pt)
* 6 How many of the tasks (completed or not) have been commented? (0, 0.5, or 1pt)
* 7 Have been the conclusions from performing the tasks clearly stated? (0, 0.5, or 1pt)
* 8 Have been the overall organization of the submitted Lab notebook been neat and easy to follow by the grader? (0, or 0.5pt) 


#### 1 Have all commands included in a raw notebook been evaluated? (0 or 0.5pt)

#### Grader's comment (if desired): 
N/A
#### Grader's comment (if desired): 
N/A

#### 2 Have all commands been experimented with? (0 or 0.5pt)

#### Grader's comment (if desired): 
N/A

#### 3 Have all experiments been briefly commented? (0 or 0.5pt)

#### Grader's comment (if desired): 
N/A

#### 4 Have all tasks been attempted? (0, 0.5, or 1pt)

#### Grader's comment (if desired): 
N/A

#### 5 How many of the tasks have been completed? (0, 0.5, or 1pt)

#### Grader's comment (if desired): 
N/A

#### 6 How many of the tasks (completed or not) have been commented? (0, 0.5, or 1pt)

#### Grader's comment (if desired): 
N/A

#### 7 Have been the conclusions from performing the tasks clearly stated? (0, 0.5, or 1pt)

#### Grader's comment (if desired): 
N/A

#### 8 Have been the overall organization of the submitted Lab notebook been neat and easy to follow by the grader? (0, or 0.5pt)

#### Grader's comment (if desired): 
N/A

### Overall score

### Score and grader's comment (if desired): 
N/A